第十三章 正确重载运算符

13.1 运算符重载基础

In [1]:
#对于python
#不能重载内置类型的运算符
#不能新建运算符，只能重载现有的
#某些运算符不能重载——is,and,not,or(不过位运算符&、|和~可以）

13.2 一元运算符

In [2]:
x=2 #取负运算符 - (__neg__)
-x

-2

In [3]:
+x #取正运算符 +(__pos__)

2

In [8]:
~x #~x==-(x+1) 对整数按位取反  ~ (__invert__)

-3

In [9]:
~4 

-5

支持一元运算符只需实现相应的特殊方法。这些特殊方法只有一个参数self，且始终返回一个新对象。

In [1]:
#10.6 中Vector类的定义
 #添加__hash__方法，与__eq__方法一起使用
from array import array
import reprlib
import math
import numbers
import functools
import operator

#序列类型的构造方法最后接受可迭代的对象为参数，因为所有内置的序列类型都是这样
class Vector:
    typecode='d'
    shotcut_names = 'xyzt'
    
    def __getattr__(self, name):
        cls=type(self)
        if len(name)==1:
            pos=cls.shotcut_names.find(name)
            if 0<=pos<len(self._component):
                return self._component[pos]
        msg='{.__name__!r} object has no arrtibute {!r}'
        raise AttributeError(msg.format(cls,name))
        
    
    def __init__(self,components):
        self._component = array(self.typecode,components)
    
    def __iter__(self):
        return iter(self._component)
    
    def __repr__(self):
        components = reprlib.repr(self._component)#reprlib模块可以生成长度有限的表示形式
        components = components[components.find('['):-1]#把字符串插入Vector的构造方法调用之前，去掉前面arra('d'和后面的)
        return 'Vector({})'.format(components)
    
    def __str__(self):
        return str(tuple(self))
    
    def __bytes__(self):
        return (bytes[ord(self.typecode)])+bytes(self._component)
    
    def __eq__(self, other):
        return tuple(self)==tuple(other)
    
    def __abs__(self):
        return math.sqrt(sum(x*x for x in self))
    
    
    def __bool__(self):
        return bool(abs(self))
    
    @classmethod
    def frombytes(cls,octets):
        typecode=chr(octets[0])
        memv = memoryview(octets[1:]).cast(typecode)
        return cls(memv)#不用像上一章一样使用*拆包，因为Vector是可迭代的
    
    def __len__(self):
        return len(self._component)
    
    def __getitem__(self, index):
        cls=type(self)
        if isinstance(index,slice):
            return cls(self._component[index])
        elif isinstance(index,numbers.Integral):
            return self._component[index]
        else:
            msg='{cls.__name__} indices must be itegers'
            raise TypeError(msg.format(cls=cls))  
      
     #注意，我们没有禁止为所有全部属性赋值，只是禁止为单个小写字母属性赋值，以防与只读属性x,y,z,t混淆   
    def __setattr__(self, name, value):
        cls = type(self)
        if len(name) == 1:  # <1>
            if name in cls.shotcut_names:  # <2>
                error = 'readonly attribute {attr_name!r}'
            elif name.islower():  # <3>
                error = "can't set attributes 'a' to 'z' in {cls_name!r}"
            else:
                error = ''  # <4>
            if error:  # <5>
                msg = error.format(cls_name=cls.__name__, attr_name=name)
                raise AttributeError(msg)
        super().__setattr__(name, value)  # <6>
        
    def __hash__(self):
        hashes=(hash(x) for x in self._component)
        return functools.reduce(operator.xor,hashes,0)#0是初始值
    
    
    
    #把一元运算符-、+添加到Vector类当中
    def __abs__(self):
        return math.sqrt(sum(x*x for x in self))
    
    def __neg__(self):
        return Vector(-x for x in self)#新构建了一个实例
    
    def __pos__(self):
        return Vector(self)#同上


In [2]:
# x和+x不相等的情况
import decimal
ctx=decimal.getcontext()
ctx.prec=40
one_third=decimal.Decimal('1')/decimal.Decimal('3')
one_third

Decimal('0.3333333333333333333333333333333333333333')

In [20]:
one_third==+one_third

True

In [3]:
ctx.prec=28
one_third==+one_third

False

In [4]:
one_third

Decimal('0.3333333333333333333333333333333333333333')

In [5]:
+one_third

Decimal('0.3333333333333333333333333333')

In [6]:
import  collections
ct=collections.Counter('abracadabra')
ct

Counter({'a': 5, 'b': 2, 'c': 1, 'd': 1, 'r': 2})

In [7]:
ct['r']=-3
ct['d']=0
ct

Counter({'a': 5, 'b': 2, 'c': 1, 'd': 0, 'r': -3})

In [8]:
+ct

Counter({'a': 5, 'b': 2, 'c': 1})

13.3 重载向量加法运算符+

In [9]:
#运用猴子补丁添加Vector.__add__方法
import itertools
def add(self,other):
    pairs=itertools.zip_longest(self,other,fillvalue=0.0)
    return Vector(a+b for a,b in pairs)

Vector.__add__=add 

In [10]:
v1=Vector([3,4,5])
v1+(10,20,30)

Vector([13.0, 24.0, 35.0])

In [11]:
v1+(1,2)

Vector([4.0, 6.0, 5.0])

In [12]:
#如果对调操作数，混合类型的加法就会失败
(10,20,30)+v1

TypeError: can only concatenate tuple (not "Vector") to tuple

![chapter13-3](image/chapter13-3.png)

In [18]:
#添加__radd__方法
def radd(self,other):
    return self+other
Vector.__radd__=radd 

In [19]:
(12,2,3)+v1

Vector([15.0, 6.0, 8.0])

In [20]:
#Vector.__add__方法的操作数要求是可迭代对象
v1+1

TypeError: unsupported operand type(s) for +: 'Vector' and 'int'

In [21]:
#Vector.__add__方法的操作数要求是可迭代的数值对象
v1+'ABC'

TypeError: unsupported operand type(s) for +: 'Vector' and 'str'

![chapter13-3-2](image/chapter13-3-2.png)

In [22]:
#实现+运算符的最终版
def add(self,other):
    try:
        pairs=itertools.zip_longest(self,other,fillvalue=0.0)
        return Vector(a+b for a,b in pairs)
    except TypeError:
        return NotImplemented
Vector.__add__=add 
#这样就算有TypeError，也能将它捕获后返回NotImplemented,再尝试调用反向运算符方法

13.4 重载标量乘法运算符*

In [24]:
def mul(self,scalar):
    return Vector(n*scalar for n in self)
def rmul(self,scalar):
    return self*scalar
Vector.__mul__=mul 
Vector.__rmul__=rmul

In [25]:
v1*2

Vector([6.0, 8.0, 10.0])

In [26]:
3*v1

Vector([9.0, 12.0, 15.0])

![chapter13-4](image/chapter13-4.png)

![chapter13-4-2](image/chapter13-4-2.png)

13.5 众多比较运算符

![chapter13-5](image/chapter13-5.jpg)

In [31]:
va=Vector([1,2,3])

In [33]:
va==Vector(range(1,4))

True

In [34]:
va==[1,2,3]

False

In [35]:
va==(1,2,3)

False

In [36]:
#改进Vector类的__eq__方法
def eq(self,other):
    if isinstance(other,Vector):
        return (len(self)==len(other) and
                all(a==b for a,b in zip(self,other)))
    else:
        return NotImplemented

In [37]:
Vector.__eq__=eq 

In [38]:
va==[1,2,3]

False

In [39]:
va==(1,2,3)

False

In [40]:
vb=Vector(range(1,4))

In [42]:
va !=vb #不需实现!=运算符，因为从object继承的__ne__方法的后备行为满足了我们的需求：定义了__eq__方法，而且它不返还
        #NotImplemented,__ne__会对__eq__返回的结果取反

False

In [43]:
#从object继承的__ne__方法运作方式类似如下，不过原版是用c语言实现的
def __ne__(self,other):
    eq_result=self==other
    if eq_result is NotImplemented:
        return NotImplemented
    else:
        return not eq_result

13.6 增量赋值运算符

In [44]:
v1=Vector([1,2,3])
v1_alias=v1
id(v1)

2654553807840

In [45]:
v1+=Vector([4,5,6])
v1

Vector([5.0, 7.0, 9.0])

In [46]:
id(v1)

2654553807896

In [47]:
v1_alias

Vector([1.0, 2.0, 3.0])

In [48]:
v1 *= 11
v1

Vector([55.0, 77.0, 99.0])

In [49]:
id(v1)

2654553804872

对于不可变类型，a+=b和a=a+b完全一样

In [55]:
import sys
sys.path.insert(0,r'D:\WORKSPACE2\python35\python8.25\fluent_python\chapter11')

In [56]:
import itertools  # <1>

from tombola import Tombola
from bingo import BingoCage


class AddableBingoCage(BingoCage):  # <2>
    #调用AddableBingoCage构造方法创建一个新实例作为结果返回
    def __add__(self, other):
        if isinstance(other, Tombola):  # <3>
            return AddableBingoCage(self.inspect() + other.inspect())  # <6>
        else:
            return NotImplemented
    
    #+=是把修改后的self返回
    def __iadd__(self, other):
        if isinstance(other, Tombola):
            other_iterable = other.inspect()  # <4>
        else:
            try:
                other_iterable = iter(other)  # <5>
            except TypeError:  # <6>
                self_cls = type(self).__name__
                msg = "right operand in += must be {!r} or an iterable"
                raise TypeError(msg.format(self_cls))
        self.load(other_iterable)  # <7>
        return self  # <8>


In [57]:
vowel='AEIOU'
globe=AddableBingoCage(vowel)
globe.inspect()

('A', 'E', 'I', 'O', 'U')

In [58]:
globe.pick()

'E'

In [59]:
len(globe.inspect())

4

In [62]:
globe2=AddableBingoCage('XYZ')
globe3=globe+globe2
len(globe3.inspect())

7

In [63]:
void=globe+[10,20]

TypeError: unsupported operand type(s) for +: 'AddableBingoCage' and 'list'

In [64]:
globe_orig = globe
len(globe.inspect())

4

In [65]:
globe+=globe2
len(globe.inspect())

7

In [66]:
globe+=['M','N']#+=的右操作数可以是任何可迭代对象
len(globe.inspect())

9

In [67]:
globe is globe_orig#globe始终指代globe_orig对象

True

In [68]:
globe+=1 #不能与非可迭代对象相加

TypeError: right operand in += must be 'AddableBingoCage' or an iterable